In [2]:
from datetime import datetime

import pandas as pd

In [3]:
calendar_df = pd.read_csv("./data/m5-forecasting-accuracy/calendar.csv")
sell_prices_df = pd.read_csv("./data/m5-forecasting-accuracy/sell_prices.csv")
sales_train_validation_df = pd.read_csv("./data/m5-forecasting-accuracy/sales_train_validation.csv")
sales_train_evaluation_df = pd.read_csv("./data/m5-forecasting-accuracy/sales_train_evaluation.csv")

In [4]:
# note: "d" is an encoding of "days since 2011-01-28"
assert ((pd.to_datetime(calendar_df["date"]) - datetime(2011, 1, 28)).dt.days.map(lambda n: f"d_{n}") == calendar_df["d"]).all()

In [5]:
sales_unstacked = sales_train_evaluation_df.drop(columns=["item_id", "dept_id", "cat_id", "store_id", "state_id"]).set_index("id").unstack().reset_index()
sales_unstacked.columns = ["day", "id", "count"]
sales_unstacked["date"] = datetime(2011, 1, 28) + pd.to_timedelta(sales_unstacked["day"].str.removeprefix("d_").astype(int), unit="days")

In [ ]:
sales_joined = pd.merge(sales_unstacked, sales_train_evaluation_df[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]], on="id")
sales_joined = pd.merge(sales_joined, calendar_df[["d", "wm_yr_wk"]].rename(columns={"d": "day"}), on="day")
sales_joined = pd.merge(sales_joined, sell_prices_df, on=["item_id", "store_id", "wm_yr_wk"], how="left")
# ensure we still have the same rows
assert len(sales_unstacked) == len(sales_joined)

In [ ]:
# convenience columns
sales_joined["year"] = sales_joined["date"].dt.year
# faster version of sales_joined["month"] = sales_joined["date"].dt.strftime("%Y%m")
sales_joined["month"] = sales_joined["date"].dt.year * 100 + sales_joined["date"].dt.month

In [ ]:
sales_joined.head()

,day,id,count,date,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,sell_price,year,month
0,d_1,HOBBIES_1_001_CA_1_evaluation,0,2011-01-29,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,11101,NaN,2011,201101
1,d_1,HOBBIES_1_002_CA_1_evaluation,0,2011-01-29,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,11101,NaN,2011,201101
2,d_1,HOBBIES_1_003_CA_1_evaluation,0,2011-01-29,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,11101,NaN,2011,201101
3,d_1,HOBBIES_1_004_CA_1_evaluation,0,2011-01-29,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,11101,NaN,2011,201101
4,d_1,HOBBIES_1_005_CA_1_evaluation,0,2011-01-29,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,11101,NaN,2011,201101


In [ ]:
sales_joined.to_parquet("./data/m5-forecasting-accuracy/sales-joined.parq.brotli", compression='brotli')

In [ ]:
for year, subdf in sales_joined.groupby("year"):
    subdf.to_parquet(f"./data/m5-forecasting-accuracy/sales-joined-{year}.parq.brotli", compression='brotli')